In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import lightgbm as lgb


from path_setup import setup_paths
setup_paths()

In [2]:
from dataset_transformer import BaseDatasetTransform
from base_model_train import BaseModelTrain
from model_evaluator import ModelEvaluator
from embeddings_tools import get_embeddings
from intelligence_tools import AnalyzeVariables

In [3]:
df = pd.read_csv('../datasets/adult.csv')

In [5]:
av = AnalyzeVariables(df)

In [7]:
av.shape()

'32561 - строк и 15 колонок с переменными в датасете'

In [10]:
av.feature_imortance(target=['income'])

c:\Users\Dmitriy.Burnashev\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(        workclass  education  marital.status  occupation  relationship  \
 income   0.179208   0.368838        0.447404    0.351892      0.453585   
 
             race       sex  native.country  
 income  0.100812  0.215904        0.098705  ,
              age    fnlwgt  education.num  capital.gain  capital.loss  \
 income  0.295797  0.030136       0.415646      1.173496      0.141095   
 
         hours.per.week  
 income        0.252488  )

In [11]:
bdf = BaseDatasetTransform(df,target='income')

In [12]:
X_train, X_test, y_train, y_test,categorical_features = bdf.fit_transform()

Series([], dtype: int64)

В колонке(ах) ['income'] нет пропущенных значений
В наборе данных нет пропущенных значений
-------------------------------------------
Информация о колонках в датасете

Категориальные колонки:
Index(['workclass', 'education', 'marital.status', 'occupation',
       'relationship', 'race', 'sex', 'native.country'],
      dtype='object')

Числовые колонки:
Index(['age', 'fnlwgt', 'education.num', 'capital.gain', 'capital.loss',
       'hours.per.week'],
      dtype='object')


In [14]:
lgbm_model = lgb.LGBMClassifier(verbose=-1,is_balanced=True)



columns_to_delete = []  


model_train = BaseModelTrain(
    model=lgbm_model,
    columns_to_delete=columns_to_delete,
    categorical_features=categorical_features,
    boosting_type='gbdt',
    objective='binary',
    metric='binary_error'
)

In [15]:
metrics = ['accuracy', 'precision', 'recall', 'f1_macro']
evaluator = ModelEvaluator(model_trainer=model_train, metrics=metrics)

evaluator.fit(X_train, y_train)

c:\Users\Dmitriy.Burnashev\AppData\Local\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Dmitriy.Burnashev\AppData\Local\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


ModelEvaluator(metrics=['accuracy', 'precision', 'recall', 'f1_macro'],
               model_trainer=BaseModelTrain(categorical_features=['workclass',
                                                                  'education',
                                                                  'marital.status',
                                                                  'occupation',
                                                                  'relationship',
                                                                  'race', 'sex',
                                                                  'native.country'],
                                            columns_to_delete=[],
                                            model=LGBMClassifier(is_balanced=True,
                                                                 metric='binary_error',
                                                                 objective='binary',
                                                                 verbose=-1)))

In [16]:
evaluator.evaluate_to_dataframe(X_test, y_test)

,Metric,Score
0,accuracy,0.870611
1,precision,0.832676
2,recall,0.799814
3,f1,0.814134
